# Classification on FIFA Dataset Using Keras



## 1. Data Exploration

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                18207 non-null  int64  
 1   ID                        18207 non-null  int64  
 2   Name                      18207 non-null  object 
 3   Age                       18207 non-null  int64  
 4   Photo                     18207 non-null  object 
 5   Nationality               18207 non-null  object 
 6   Flag                      18207 non-null  object 
 7   Overall                   18207 non-null  int64  
 8   Potential                 18207 non-null  int64  
 9   Club                      17966 non-null  object 
 10  Club Logo                 18207 non-null  object 
 11  Value                     18207 non-null  object 
 12  Wage                      18207 non-null  object 
 13  Special                   18207 non-null  int64  
 14  Prefer

In [4]:
# Remove Missing Values 
na = pd.notnull(df["Position"])
df = df[na]

## 2. Feature Selection and Data Transformation

In [5]:
df = df[["Position", 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes']]

In [6]:
df.head()

,Position,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,RF,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,ST,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,LW,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,GK,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,RCM,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [7]:
forward_player = ["ST", "LW", "RW", "LF", "RF", "RS","LS", "CF"]
midfielder_player = ["CM","RCM","LCM", "CDM","RDM","LDM", "CAM", "LAM", "RAM", "RM", "LM"]
defender_player = ["CB", "RCB", "LCB", "LWB", "RWB", "LB", "RB"]

In [8]:
df.loc[df["Position"] == "GK", "Position"] = 0

df.loc[df["Position"].isin(defender_player), "Position"] = 1

df.loc[df["Position"].isin(midfielder_player), "Position"] = 2

df.loc[df["Position"].isin(forward_player), "Position"] = 3

In [9]:
x = df.drop("Position", axis = 1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = pd.DataFrame(sc.fit_transform(x))
y = df["Position"]

In [10]:
from keras.utils.np_utils import to_categorical
y_cat = to_categorical(y)

## 3. Training Keras Models

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2)

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K

model = Sequential()
model.add(Dense(60, input_shape = (33,), activation = "relu"))
model.add(Dense(15, activation = "relu"))
#model.add(Dropout(0.2))
model.add(Dense(4, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 60)                2040      
                                                                 
 dense_3 (Dense)             (None, 15)                915       
                                                                 
 dense_4 (Dense)             (None, 4)                 64        
                                                                 
Total params: 3,019
Trainable params: 3,019
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [17]:
model.fit(x_train, y_train, verbose=1, epochs=10)


Epoch 1/10
454/454 [==============================] - 3s 3ms/step - loss: 0.3486 - accuracy: 0.8529
Epoch 2/10
454/454 [==============================] - 1s 2ms/step - loss: 0.3083 - accuracy: 0.8693
Epoch 3/10
454/454 [==============================] - 1s 2ms/step - loss: 0.3062 - accuracy: 0.8726
Epoch 4/10
454/454 [==============================] - 1s 2ms/step - loss: 0.2930 - accuracy: 0.8798
Epoch 5/10
454/454 [==============================] - 1s 2ms/step - loss: 0.2939 - accuracy: 0.8802
Epoch 6/10
454/454 [==============================] - 1s 2ms/step - loss: 0.2912 - accuracy: 0.8788
Epoch 7/10
454/454 [==============================] - 1s 2ms/step - loss: 0.2886 - accuracy: 0.8809
Epoch 8/10
454/454 [==============================] - 1s 2ms/step - loss: 0.2850 - accuracy: 0.8830
Epoch 9/10
454/454 [==============================] - 1s 2ms/step - loss: 0.2836 - accuracy: 0.8806
Epoch 10/10
454/454 [==============================] - 1s 2ms/step - loss: 0.2786 - accuracy: 0.8828

## 4. Model Evaluation

In [36]:
from sklearn.metrics import confusion_matrix

In [37]:
y_pred_class = np.argmax(model.predict(x_test),axis=-1)

In [38]:
y_test_class = np.argmax(y_test, axis=1)

In [39]:
y_test_class

array([2, 1, 1, ..., 1, 0, 3])

In [40]:
confusion_matrix(y_test_class, y_pred_class)

array([[ 418,    0,    0,    0],
       [   0, 1007,  133,    0],
       [   0,   72, 1263,   67],
       [   0,    3,  169,  498]])

In [41]:
from sklearn.metrics import classification_report

In [43]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       418
           1       0.93      0.88      0.91      1140
           2       0.81      0.90      0.85      1402
           3       0.88      0.74      0.81       670

    accuracy                           0.88      3630
   macro avg       0.90      0.88      0.89      3630
weighted avg       0.88      0.88      0.88      3630

